# GONZAG example

This notebook demonstrates how you can blablabla

# Run the script to generate the figures

In order to generate the figures you need 1 netCDF file per bulk algorithm tested. Of the form:

`STATION_ASF-<ALGO>_1h_<YEAR1>0101_<YEAR2>1231_gridT.nc`


Example, in the default setup, for the "COARE 3.6" bulk algorithm, the following file is generated:

`STATION_ASF-COARE3p6_1h_20180101_20181231_gridT.nc`

These files are to be found in the subdirectory `output` of the production directory of `STATION_ASF`, namely:
- `${PROD_DIR}/output/` into production script `launch_sasf.sh`
- `cprod_dir+'/output/'` into this notebook

## Start:

In [8]:
import sys
from os import getenv
sys.path.append("../")

import gonzag as gz
from gonzag.config import ldebug


GONZAG_DATA_DIR = '/MEDIA/data/GONZAG/conzag_input'
CLIMPORN_DIR    = getenv('HOME')+'/DEV/climporn/python' ; print(CLIMPORN_DIR)



/home/laurent/DEV/climporn/python
